In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
# 加载任务1处理过的数据集
pickle_file = 'E:/Python/data/notMNIST.pickle'
with open(pickle_file, 'rb') as f:  
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    # reshape(-1, image_size * image_size)代表你只知道数组的第二个维度，
    # 剩下的维度由Numpy自动计算得出，比如原本维度（20000*28*28）train_dataset
    # reshape过后的维度就是20000*(28*28)
    # test_dataset的维度就是10000*(28*28)
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # 将图片标签映射为数组形式，比如1映射为[False, True, False, ..., False, False, False]
    labels = (np.arange(num_labels)) == labels[:, None].astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# np.argmax()返回沿着轴线方向最大值的索引值
# 判断预测标签与原标签是否相同，并计算相同标签的数目，除以总标签数，即正确率
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# 一、Multinomial logistic regression with gradient descent

首先，我们用简单的梯度下降方法训练一个多分类逻辑斯蒂模型，通过这个部分，你将会了解到tensorflow工作“模板”。

tensorflow是这样工作的：

- 告诉它计算要求：输入（inputs）、变量（variables）、运算方式（operations looks like），然后tensorflow就会在计算图（computationa graph）中建立一个节点

    - with graph_as_default(): 

- 然后你就可以根据自己的需要在计算图上通过多次调用session.run()来进行计算

    - withtf.Session(graph=graph) as session
    
给逻辑斯蒂和神经网络模型引入可调节的L2正则化参数。在TensorFlow中，你可以使用nn.l2_loss(t)给张量计算L2损失，合适的正则化系数会给模型的验证集、测试集准确度带来提升。

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
# 先取10000的样本量
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # Input data.（输入）
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.（变量）
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    # 随机初始化权重矩阵（服从正态分布），偏置项初始化为0
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    # 注意正确的方式应为tf.nn.softmax_cross_entropy_with_logits(logits=..., labels=...)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    # 梯度下降算法优化：tf.train.GradientDescentOptimizer()，找到最小损失
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    # 根据上面的做法初始化所有变量
    tf.initialize_all_variables().run()
    print('初始化变量...')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            %time print('迭代次数到 %d 时损失为: %f' % (step, l))
            %time print('训练集准确率: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
    # Calling .eval() on valid_prediction is basically like calling run(), but
    # just to get that one numpy array. Note that it recomputes all its graph
    # dependencies.
            %time print('验证集准确率: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    %time print('测试集准确率: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
初始化变量...
迭代次数到 0 时损失为: 16.769234
Wall time: 486 µs
训练集准确率: 6.9%
Wall time: 990 µs
验证集准确率: 9.8%
Wall time: 87.8 ms
迭代次数到 100 时损失为: 2.401129
Wall time: 501 µs
训练集准确率: 70.8%
Wall time: 501 µs
验证集准确率: 69.8%
Wall time: 2.01 ms
迭代次数到 200 时损失为: 1.937966
Wall time: 0 ns
训练集准确率: 74.0%
Wall time: 501 µs
验证集准确率: 72.3%
Wall time: 2.01 ms
迭代次数到 300 时损失为: 1.680977
Wall time: 0 ns
训练集准确率: 75.3%
Wall time: 501 µs
验证集准确率: 73.2%
Wall time: 2.01 ms
迭代次数到 400 时损失为: 1.506529
Wall time: 0 ns
训练集准确率: 76.3%
Wall time: 501 µs
验证集准确率: 73.7%
Wall time: 2.01 ms
迭代次数到 500 时损失为: 1.376693
Wall time: 0 ns
训练集准确率: 77.1%
Wall time: 502 µs
验证集准确率: 73.9%
Wall time: 1.5 ms
迭代次数到 600 时损失为: 1.274513
Wall time: 0 ns
训练集准确率: 77.6%
Wall time: 502 µs
验证集准确率: 74.0%
Wall time: 2.01 ms
迭代次数到 700 时损失为: 1.191335
Wall time: 0 ns
训练集准确率: 78.0%
Wall time: 502 µs
验证集准确率: 74.3%
Wall time: 2.51 ms
迭代次数到 800 时损失为: 1.122059
Wall time: 0 ns
训练集准确率: 78.4%
Wall time: 501

# 二、With SGD

用梯度下降算法优化的效果也还不错，并且速度还非常快！

现在我们转用比梯度下降方法速度更快的随机梯度下降（SGD），两种方法用到的图是相似的。

不想梯度下降方法将所有的训练数据放进一个节点，SGD建立了一个placeholder节点，每次运行session.run()的时候都会分配一个batch size的数据给placeholder。

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    print(biases.get_shape())

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    print(logits.get_shape())
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

(10,)
(128, 10)


In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("初始化变量...")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch 准确率: %.1f%%" % accuracy(predictions, batch_labels))
            print("验证集准确率: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("测试集准确率: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
初始化变量...
Minibatch loss at step 0: 16.593002
Minibatch 准确率: 9.4%
验证集准确率: 10.6%
Minibatch loss at step 500: 1.991482
Minibatch 准确率: 67.2%
验证集准确率: 75.3%
Minibatch loss at step 1000: 1.248387
Minibatch 准确率: 77.3%
验证集准确率: 76.5%
Minibatch loss at step 1500: 1.164837
Minibatch 准确率: 78.1%
验证集准确率: 77.3%
Minibatch loss at step 2000: 0.666750
Minibatch 准确率: 87.5%
验证集准确率: 77.5%
Minibatch loss at step 2500: 0.749796
Minibatch 准确率: 84.4%
验证集准确率: 78.2%
Minibatch loss at step 3000: 1.067075
Minibatch 准确率: 76.6%
验证集准确率: 78.1%
测试集准确率: 85.8%


# 三、ReLUs 网络

将运用SGD的LR回归例子转换成只有一个隐藏层（1024个隐藏层节点）的ReLUs神经网络。经比较，这个模型能够提升验证集和测试集精度，性能有了大幅度提升。

In [9]:
batch_size = 128
relu_count = 10000 #隐藏节点个数

graph = tf.Graph()
with graph.as_default():
    # 输入数据
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # 变量
    # note：这里考到了神经网络中练习参数（权重和偏置项）个数的计算
    # see：https://classroom.udacity.com/courses/ud730/lessons/6379031992/concepts/63789029420923
    # 输入层→隐藏层：权重个数1024*1*1*(28*28)，偏置项个数1024
    # 隐藏层→输出层：权重个数10*1*1*1024，偏置项个数10
    # 输出层：10*1
    weights = [tf.Variable(tf.truncated_normal([image_size * image_size, relu_count])),
              tf.Variable(tf.truncated_normal([relu_count, num_labels]))]
    biases = [tf.Variable(tf.zeros([relu_count])),
              tf.Variable(tf.zeros([num_labels]))]
    
    # 训练计算
    hidden_layer = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights[0]), biases[0]))
    logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights[0]), biases[0]))
                                                      , weights[1]), biases[1]))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights[0]), biases[0]))
                                                      , weights[1]), biases[1]))

In [10]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 5000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("初始化变量...")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch 准确率: %.1f%%" % accuracy(predictions, batch_labels))
            print("验证集准确率: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("测试集准确率: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
初始化变量...
Minibatch loss at step 0: 1234.318726
Minibatch 准确率: 10.2%
验证集准确率: 19.1%
Minibatch loss at step 500: 254.571014
Minibatch 准确率: 78.1%
验证集准确率: 82.4%
Minibatch loss at step 1000: 205.607880
Minibatch 准确率: 81.2%
验证集准确率: 83.1%
Minibatch loss at step 1500: 36.849697
Minibatch 准确率: 89.1%
验证集准确率: 83.7%
Minibatch loss at step 2000: 20.069494
Minibatch 准确率: 93.8%
验证集准确率: 85.3%
Minibatch loss at step 2500: 36.324703
Minibatch 准确率: 91.4%
验证集准确率: 84.4%
Minibatch loss at step 3000: 20.559185
Minibatch 准确率: 90.6%
验证集准确率: 85.4%
Minibatch loss at step 3500: 5.260818
Minibatch 准确率: 92.2%
验证集准确率: 86.3%
Minibatch loss at step 4000: 46.144531
Minibatch 准确率: 88.3%
验证集准确率: 86.1%
Minibatch loss at step 4500: 14.380877
Minibatch 准确率: 89.8%
验证集准确率: 85.0%
测试集准确率: 92.6%
